In [ ]:
!pip install -q wordcloud emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import emoji
from wordcloud import WordCloud, STOPWORDS
import re
import nltk
from nltk.corpus import stopwords #corpora
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df=pd.read_csv("Tesla.csv")
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1546541426317590528,1545826164564000768,1.657559e+12,2022-07-11 17:06:24,0,NaN,@GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'GailAlfarATX', 'name': 'Gail...",NaN,NaN,NaN,NaN
1,1,1546541415857102850,1545826164564000768,1.657559e+12,2022-07-11 17:06:21,0,NaN,@elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'elonmusk', 'name': 'Elon Mus...",NaN,NaN,NaN,NaN
2,2,1546541411897581568,1546541411897581568,1.657559e+12,2022-07-11 17:06:20,0,NaN,"@elonmusk #Think about buying a country , #Mex...",en,"['think', 'mexico', 'constitution']",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,3,1546541379110805508,1546340000500813824,1.657559e+12,2022-07-11 17:06:12,0,NaN,"@get_innocuous Actual receipts, and yet you ha...",en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'get_innocuous', 'name': 'tra...",NaN,NaN,NaN,NaN
4,4,1546541363470028800,1546541363470028800,1.657559e+12,2022-07-11 17:06:09,0,NaN,Tesla wall battery for the save! Power went ou...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
df=df[['created_at','tweet']]
df.head()

,created_at,tweet
0,1.657559e+12,@GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...
1,1.657559e+12,@elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...
2,1.657559e+12,"@elonmusk #Think about buying a country , #Mex..."
3,1.657559e+12,"@get_innocuous Actual receipts, and yet you ha..."
4,1.657559e+12,Tesla wall battery for the save! Power went ou...


In [ ]:
df.shape

(10016, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10016 entries, 0 to 10015
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   created_at  10016 non-null  float64
 1   tweet       10016 non-null  object 
dtypes: float64(1), object(1)
memory usage: 156.6+ KB


In [ ]:
df.isna().sum()

,0
created_at,0
tweet,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
# i don't have car
# i do not have car


contractions = {"ain't": "am not", "aren't": "are not", "can't": "cannot",
"can't've": "cannot have", "'cause": "because", "could've": "could have",
"couldn't": "could not", "couldn't've": "could not have", "didn't": "did not",
"doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have",
"he'll": "he will", "he's": "he is", "how'd": "how did", "how'll": "how will",
"how's": "how is", "i'd": "i would", "i'll": "i will", "i'm": "i am", "i've": "i have",
"isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is",
"let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
"mightn't": "might not", "must've": "must have", "mustn't": "must not",
"needn't": "need not", "oughtn't": "ought not", "shan't": "shall not",
"sha'n't": "shall not", "she'd": "she would", "she'll": "she will", "she's": "she is",
"should've": "should have", "shouldn't": "should not", "that'd": "that would",
"that's": "that is", "there'd": "there had", "there's": "there is", "they'd": "they would",
"they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not",
"we'd": "we would", "we'll": "we will", "we're": "we are", "we've": "we have",
"weren't": "were not", "what'll": "what will", "what're": "what are", "what's": "what is",
"what've": "what have", "where'd": "where did", "where's": "where is", "who'll": "who will",
"who's": "who is", "won't": "will not", "wouldn't": "would not", "you'd": "you would",
"you'll": "you will", "you're": "you are", "wfh": "work from home", "wfo": "work from office",
"idk": "i do not know", "brb": "be right back", "btw": "by the way", "tbh": "to be honest",
"omw": "on my way", "lmk": "let me know", "fyi": "for your information",
"imo": "in my opinion", "smh": "shaking my head", "nvm": "never mind",
"ikr": "i know right", "fr": "for real", "rn": "right now", "gg": "good game",
"dm": "direct message", "afaik": "as far as i know", "bff": "best friends forever",
"ftw": "for the win", "hmu": "hit me up", "ggwp": "good game well played"}


# Text data Cleaning

In [ ]:
text= "I don't have a supercar! 😎 Visit https://cars.com now!"

## Step 1: Lowercase the text


In [ ]:
text = text.lower()
text

"i don't have a supercar! 😎 visit https://cars.com now!"

## Step 2: Handle contractions


In [ ]:
text = text.split()
text


['i',
 "don't",
 'have',
 'a',
 'supercar!',
 '😎',
 'visit',
 'https://cars.com',
 'now!']

In [ ]:
new_text = []
for word in text:
    if word in contractions:
        new_text.append(contractions[word])
    else:
        new_text.append(word)

new_text

['i',
 'do not',
 'have',
 'a',
 'supercar!',
 '😎',
 'visit',
 'https://cars.com',
 'now!']

In [ ]:
text = " ".join(new_text)
text

'i do not have a supercar! 😎 visit https://cars.com now!'

## Step 3: Remove URLs

In [ ]:
text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
text

'i do not have a supercar! 😎 visit '

## Step 4: Remove usernames (e.g., @user)

In [ ]:
text = re.sub(r'@[A-Za-z0-9]+', '', text)
text

'i do not have a supercar! 😎 visit '

## Step 5: Remove HTML and symbols

In [ ]:
text = re.sub(r'\<a href', ' ', text)
text = re.sub(r'&amp;', '', text)
text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
text = re.sub(r'<br />', ' ', text)
text = re.sub(r'\'', ' ', text)

##  Step 6: Remove emojis

In [ ]:
emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF" \
                               "\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF" \
                               "\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF" \
                               "\U00002702-\U000027B0\U000024C2-\U0001F251]+", flags=re.UNICODE)
text = emoji_pattern.sub(r'', text)
text

'i do not have a supercar   visit '

## Step 7: Tokenization


In [ ]:
words = word_tokenize(text)
words

['i', 'do', 'not', 'have', 'a', 'supercar', 'visit']

##  Step 8: Remove stopwords

In [ ]:
stop_words = set(stopwords.words("english"))
words = [word for word in words if word not in stop_words]
words

['supercar', 'visit']

## Step 9: Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words]
words

['supercar', 'visit']

# Text data cleaning function

In [ ]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    if True:
        #text= "i dont't have supercar"
        text = text.split()
        # ['i','don't','have','supercar']
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
                # ['i','do not','have','supercar']
            else:
                new_text.append(word)
        text = " ".join(new_text)
        #[i do not have supercar]

    # Remove URLs
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

    # Remove usernames
    text = re.sub(r'@[A-Za-z0-9]+', '', text)

    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    # Remove emojis
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF" \
                               "\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF" \
                               "\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF" \
                               "\U00002702-\U000027B0\U000024C2-\U0001F251]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Tokenize text
    words = word_tokenize(text)

    # Remove stopwords if needed
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(words)

In [ ]:
df['clean_tweet'] = df['tweet'].apply(lambda x:clean_text(x))
df.head()

,created_at,tweet,clean_tweet
0,1.657559e+12,@GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...,six 4 still live home homeschooled taught self...
1,1.657559e+12,@elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...,go dozen kids… missing 3 go
2,1.657559e+12,"@elonmusk #Think about buying a country , #Mex...",think buying country mexico could turn richest...
3,1.657559e+12,"@get_innocuous Actual receipts, and yet you ha...",innocuous actual receipt yet ’ asked anyone bu...
4,1.657559e+12,Tesla wall battery for the save! Power went ou...,tesla wall battery save power went still run a...


In [ ]:
from datetime import datetime
df['created_at'] = pd.to_datetime(df['created_at'], unit='ms')
df.head()

,created_at,tweet,clean_tweet
0,2022-07-11 17:06:24,@GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...,six 4 still live home homeschooled taught self...
1,2022-07-11 17:06:21,@elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...,go dozen kids… missing 3 go
2,2022-07-11 17:06:20,"@elonmusk #Think about buying a country , #Mex...",think buying country mexico could turn richest...
3,2022-07-11 17:06:12,"@get_innocuous Actual receipts, and yet you ha...",innocuous actual receipt yet ’ asked anyone bu...
4,2022-07-11 17:06:09,Tesla wall battery for the save! Power went ou...,tesla wall battery save power went still run a...


# Exploratory Data Analysis

In [ ]:
df["Tweets_length"]= df["clean_tweet"].str.len()

In [ ]:
plt.figure(figsize=(10,3))
sns.histplot(df['Tweets_length'])
plt.title("Distribution of the length of tweets")
plt.show()

## Wordcloud

In [ ]:
all_text = ' '.join(df['clean_tweet'])

In [ ]:
# Generate word cloud
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = set(STOPWORDS),
                min_font_size = 10).generate(all_text)

plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## Most popular hashtags

In [ ]:
all_hashtags = sum(
    df['tweet'].apply(lambda x: re.findall(r"#(\w+)", x.lower())),
    []
)
all_hashtags

In [ ]:
# Frequency distribution
freq_dist = nltk.FreqDist(all_hashtags)
freq_dist

In [ ]:
# Convert to DataFrame
df_hashtags = pd.DataFrame(freq_dist.items(), columns=['Hashtag', 'Count'])
df_hashtags.head()

In [ ]:
# Top 10 hashtags
top_10 = df_hashtags.nlargest(10, 'Count')

In [ ]:
# Plot
plt.figure(figsize=(16, 5))
sns.barplot(data=top_10, x='Hashtag', y='Count')
plt.title("Top 10 Most Popular Hashtags")
plt.ylabel("Count")
plt.show()

## N Gram Analysis

In [ ]:
from nltk import ngrams
from collections import Counter

In [ ]:
# Compute bigram frequencies
bigrams = list(ngrams(all_text.lower().split(), 2))
bigram_freq = Counter(bigrams)

In [ ]:
# Create and plot top N bigrams
top_N = 20
top_bigrams = bigram_freq.most_common(top_N)
bigram_df = pd.DataFrame(top_bigrams, columns=['Bigram', 'Frequency'])

In [ ]:
bigram_df.plot(x='Bigram', y='Frequency', kind='barh', figsize=(10, 6))
plt.title(f'Top {top_N} Most Frequent Bigrams')
plt.xlabel('Bigram')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Compute trigram frequencies
trigrams = list(ngrams(all_text.lower().split(), 3))
trigram_freq = Counter(trigrams)

In [ ]:
# Create and plot top N trigrams
top_N = 20
top_trigrams = trigram_freq.most_common(top_N)
trigram_df = pd.DataFrame(top_trigrams, columns=['trigram', 'Frequency'])

In [ ]:
trigram_df.plot(x='trigram', y='Frequency', kind='barh', figsize=(10, 6))
plt.title(f'Top {top_N} Most Frequent trigrams')
plt.xlabel('trigram')
plt.ylabel('Frequency')
plt.show()

## Sentiment Analysis

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
# Initialize VADER
sia = SentimentIntensityAnalyzer()

In [ ]:
# Apply sentiment scoring
df['sentiment_score'] = df['clean_tweet'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [ ]:
df['sentiment_label'] = df['sentiment_score'].apply(
    lambda score: 'positive' if score > 0.05 else ('negative' if score < -0.05 else 'neutral')
)

In [ ]:
plt.figure(figsize=(8, 3))
sns.histplot(df['sentiment_score'], bins=20, kde=True)
plt.title('Sentiment Score Distribution')
plt.xlabel('Compound Sentiment Score')
plt.ylabel('Tweet Count')
plt.tight_layout()
plt.show()

In [ ]:
sns.countplot(data=df, x='sentiment_label', order=['positive', 'neutral', 'negative'])
plt.title('Sentiment Label Distribution')
plt.show()

## Top 10 Rare Words

In [ ]:
# Combine all cleaned tweets into one string
all_words = ' '.join(df['clean_tweet']).split()

# Count word frequencies
word_freq = Counter(all_words)


In [ ]:
# Get 10 rarest words (frequency = lowest)
rare_words = word_freq.most_common()[::-1][:10]


In [ ]:
# Create DataFrame
rare_df = pd.DataFrame(rare_words, columns=['Word', 'Frequency'])

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=rare_df, x='Frequency', y='Word', palette='magma')
plt.title('Top 10 Rare Words')
plt.xlabel('Frequency')
plt.ylabel('Words')
plt.tight_layout()
plt.show()

# Count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')


In [ ]:
# Fit and transform the clean tweets
count_matrix = count_vectorizer.fit_transform(df['clean_tweet'])

In [ ]:
# Get feature names (words)
feature_names = count_vectorizer.get_feature_names_out()
feature_names

In [ ]:
# Extract top 5 keywords by count for each tweet
top_keywords_count = []
for i in range(len(df)):
    feature_index = count_matrix[i, :].nonzero()[1]
    count_scores = zip(feature_index, [count_matrix[i, x] for x in feature_index])
    top_keywords_count.append(
        sorted([(feature_names[i], int(s)) for (i, s) in count_scores], key=lambda x: x[1], reverse=True)[:5]
    )


In [ ]:
df['top_keywords_count'] = top_keywords_count
df[['clean_tweet', 'top_keywords_count']].head(10)


# Tf-IDF vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')


In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_tweet'])


In [ ]:
# Get feature names
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_names

In [ ]:
# Get top keywords for each tweet
top_keywords = []
for i in range(len(df)):
    feature_index = tfidf_matrix[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    top_keywords.append(sorted([(feature_names[i], s) for (i, s) in tfidf_scores], key=lambda x: x[1], reverse=True)[:5])


In [ ]:
# Display top keywords
df['top_keywords'] = top_keywords
df[['clean_tweet', 'top_keywords']].head(5)

# Topic Modelling

* 1️⃣ max_df=0.95- Removes very frequent words that appear in more than 95% of the documents.
* 2️⃣ min_df=2- Removes rare words that appear in less than 2 documents.
* 3️⃣ stop_words='english'- Removes common English stopwords like "the", "is", "and", "in", etc.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Tokenize and vectorize the text data
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(df['clean_tweet'])

In [ ]:
lda_model = LatentDirichletAllocation(n_components=5, random_state=42) # Topic modelling
lda_output = lda_model.fit_transform(X)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx+1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
# Display the topics
print("Topics found via LDA:")
no_top_words = 10
display_topics(lda_model, vectorizer.get_feature_names_out(), no_top_words)


# Word2Vec (Skip gram and CBOW)

In [ ]:
!pip install --upgrade numpy gensim --no-cache-dir

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
# Convert clean tweets to tokenized format
tokenized_tweets = df['clean_tweet'].apply(word_tokenize).tolist()

In [ ]:
# Build the Word2Vec model (Skip Gram)
w2v_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=2, workers=4, sg=1)  # sg=1 for Skip-Gram, sg=0 for CBOW


In [ ]:
# Vector representation of a word
print(w2v_model.wv['happy'])

In [ ]:
# Most similar words to 'love'
print(w2v_model.wv.most_similar('love'))

In [ ]:
#CBOW
w2v_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=2, workers=4, sg=0)  # sg=1 for Skip-Gram, sg=0 for CBOW

In [ ]:
# Vector representation of a word
print(w2v_model.wv['happy'])

In [ ]:
# Most similar words to 'love'
print(w2v_model.wv.most_similar('love'))